## Setup

In [27]:
from fabrictestbed.slice_manager import SliceManager, Status
from fabric_cm.credmgr.credmgr_proxy import CredmgrProxy
import os
import json

help(SliceManager)
help(CredmgrProxy)

Help on class SliceManager in module fabrictestbed.slice_manager.slice_manager:

class SliceManager(builtins.object)
 |  SliceManager(*, cm_host: str, oc_host: str, refresh_token: str, project_name: str = 'all', scope: str = 'all')
 |  
 |  Implements User facing Control Framework API interface
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *, cm_host: str, oc_host: str, refresh_token: str, project_name: str = 'all', scope: str = 'all')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  create(self, *, slice_name: str, ssh_key: str, topology: fim.user.topology.ExperimentTopology = None, slice_graph: str = None, lease_end_time: str = None) -> Tuple[fabric_cf.orchestrator.orchestrator_proxy.Status, Union[Exception, List[fabric_cf.orchestrator.elements.reservation.Reservation]]]
 |      Create a slice
 |      @param slice_name slice name
 |      @param ssh_key SSH Key
 |      @param topology Experiment topology
 |      @param slice_graph Slice Graph st

## Check: refreshing tokens with orchestrator proxy instead of slice manager
### Create proxy:

In [28]:
import os
import json

from fabric_cm.credmgr.credmgr_proxy import CredmgrProxy

#Create a Credential Manager Proxy 
credmgr_proxy = CredmgrProxy(credmgr_host=os.environ['FABRIC_CREDMGR_HOST'])

### Create slice manager:

In [29]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']

print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

# Create Slice Manager
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, 
                             refresh_token=fabric_refresh_token, project_name='all', scope='all')

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net


### Get initial refresh token:

In [30]:
fabric_refresh_token = None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token

print("Refresh Token: {}".format(fabric_refresh_token))

Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTEZBVG43TGNJQHFSWCZLCGEZGEZBUGYYTSODGMNRGEZTGMRRWMNR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRSGQYDCNBSHAYTMOJGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ


### Get initial id token:

In [31]:
#try getting it without doing refresh
try:
    id_tok = slice_manager.get_id_token()
    print("id token: {}".format(id_tok) )
except Exception as e:
    print("Error: {}".format(e))

#try getting it with a refresh through credential manager:
try:
    Status, fabric_id_token, fabric_refresh_token = credmgr_proxy.refresh(project_name='all', scope='all', refresh_token=fabric_refresh_token)

    #fabric_id_token=refresh_res['id_token']
    #fabric_refresh_token=refresh_res['refresh_token']
    print()
    print("-----")
    print("New Refresh Token: {}\n".format(fabric_refresh_token))
    print("New ID Token: {}".format(fabric_id_token))
    %store fabric_refresh_token

except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

print()
print("-----")
#try getting it with a refresh through slice manager:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
fabric_id_token = slice_manager.get_id_token()

print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("New ID Token: {}".format(fabric_id_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token
print()
print()

id token: None

-----
New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTEZBVG43TGNJQHFSWCZLCGEZGEZBUGYYTSODGMNRGEZTGMRRWMNR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRSGQYDCNBSHAYTMOJGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

New ID Token: "(\"invalid_token\"\nerror_description=\"The token \"https://cilogon.org/oauth2/2d5773509eaeb12bd46198fcbbffdcf6?type=refreshToken&ts=1622401428169&version=v2.0&lifetime=86400000\" could not be associated with a pending flow\"\n) "

Stored 'fabric_refresh_token' (str)

-----
Exception occurred while getting tokens:"(\"invalid_token\"\nerror_description=\"The token \"https://cilogon.org/oauth2/2d5773509eaeb12bd46198fcbbffdcf6?type=refreshToken&ts=1622401428169&version=v2.0&lifetime=86400000\" could not be associated with a pending flow\"\n) "

New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTEZBVG43TGNJQHFSWCZLCGEZGEZBUGYYTSODGMNRGEZTGMRRWMNR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRSGQYDCNBSHAYTMOJGOZSXE43JN5XD25RSFYYCM3D

In [34]:
status, slices = slice_manager.slices()
print("slices: {}".format(slices))

slices: Token None must be specified
